# In-Class Activity: Using LangChain With Model Context Protocol (MCP)

**Objective:** Learn how to build and run a LangChain client and MCP server that communicate through the Model Context Protocol (MCP). You will understand how MCP enables LLMs to safely interact with external tools and data sources, and how LangChain can integrate these capabilities to create more powerful and extensible AI agents.

In this activity, you will set up an MCP environment, implement a basic MCP server exposing simple tools, and connect it to a LangChain-based client that invokes these tools through natural language prompts. You will then observe how the client sends requests, the server responds with results, and analyze how this protocol supports tool-based reasoning.

Let's get started! 🚀


## 0) Important!

This notebook provides guidance on how to set up a MCP server and run a LangChain client. However, unlike previous in-class activities, you **must use the terminal on your own local computer** to run the commands outlined below. Running them in this notebook environment will **NOT** work!

## What is Model Context Protocol (MCP)?

Model Context Protocol (MCP) is an open protocol that standardizes how applications provide tools and context to LLMs. LangChain agents can use tools defined on MCP servers using the langchain-mcp-adapters library.

## 1) Prerequisites

To get started, open a terminal window and create two tabs — one for running the server and the other for running the client.

It is good practice to create a virtual environment for installing and running code.   
Run the command below to create a virtual environment called *mcp*.

```
conda create --name mcp python=3.11
conda activate mcp
```

Once the virtual environment is activated, you will see (mcp) appear before your terminal prompt.  
For example: (mcp) username@dhcp-v025-203 ~ %

## 2) Install necessary libraries


```
pip install langchain-mcp-adapters
```

**Note:** After completing the installation, make sure to restart your terminal using:

```
source ~/.bashrc
```

And make sure you are inside the virtual environment we created:
```
conda activate mcp
```

Then, in your terminal, make sure to export the Google Gemini API key by running:
```
export GOOGLE_API_KEY=<your_gemini_api_key>
```


## 3) Main Activity

### 3.1) Creating the toolset for server.

Create a new file named math_server.py and copy paste the code below

```

from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")

```


TODO: add **6** additional functions and add to the math_server.py

See: math_server.py for additional functions.

```
subtract(a: int, b: int) -> int:
divide(a: float, b: float) -> float:
power(base: float, exponent: float) -> float:
sqrt(x: float) -> float:
average(numbers: list[float]) -> float:
factorial(n: int) -> int:
```

### 3.2) Creating the client for server.

Now we are going to create and run the client.
Create a new file named client.py and copy paste the code below


```
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import asyncio

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["math_server.py"],
)

async def run_agent():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # Get tools
            tools = await load_mcp_tools(session)

            # Create and run the agent
            agent = create_react_agent(model, tools)
            agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
            return agent_response

# Run the async function
if __name__ == "__main__":
    result = asyncio.run(run_agent())
    print(result)

```

TODO: Similar to this line of code:

See: client.py for additional test cases.

```
agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
```

Write 10 test cases to verify that the six additional math functions you added work as expected. Make sure each test includes at least three operations within a single invoke call. For example, in the statement above, there are two operations (+ and ×), which count as two operations.


### 3.3) Parsing the output generated

Now, lets write an implementation to parse the output printed from the console of client.py, retrieve the final computed answer, and print both the original math question (e.g., "what's (3 + 5) × 12?") and the final answer. Make sure to copy and paste the answers in the cell below.

TODO: Copy paste the math question and final answer for 10 test cases below:

1) "Compute (5 + 3 - 2) * 4" --> 24

2) "Calculate (10 / 2 + 3) ^ 2" --> 64

3) "What is (15 % 4 * 2) - 1?" --> 5

4) "Find result of (20 // 3 + 4) * 2" --> 20

5) "Compute sqrt(16) + 2 * 5" --> 14

6) "Evaluate (3 ^ 3 - 5) / 2" --> 11

7) "Calculate (22 % 5) + (10 // 3) + 1" --> 6

8) "What is sqrt(49) ^ 2 - 10?" --> 39

9) "Find result of (6 * 3 / 2) + 4" --> 13

10) "Compute ((8 + 4) / 2 * 3) - sqrt(9)" --> 15

## For Submission

Make sure to submit the following:

- The Week 6 In-Class Activity notebook with all TODO entries completed (week6_activity.ipynb)

- The math_server.py file containing your implemented MCP server and additional math functions

- The client.py file used to test and demonstrate the math server operations